# Notebook para creación de paradas y nodos - Notebook for stop and node creation

Este notebook es una herramienta para crear un set de paradas tal cual lo necesita `Urbantrips`, con un orden de paso y un `node_id` (que se utilizará para construir grafos. Existen 2 situaciones no excluyentes. Por un lado, se tiene una cartografía de recorridos de las diferentes lineas y ramales y se quiere establecer paradas utilizando un intervalo regular en metros a lo largo de ese recorrido. Esta situación es típica de recorridos de colectivos. Por el otro, se tiene una cartografía de paradas y una de recorridos para las diferentes lineas y ramales, para las cuales se quiere establecer un orden de paso. 

Este notebooks no es más que un insumo para ayudar en esta tareay facilitar la elaboración del dataset de paradas. El archivo final seguramente necesite ediciones customizadas y hechas a mano con otro criterio. El dataset final de paradas debe llamarse `stops.csv` y estar localizado en el directorio `data/data_ciudad/`. Si tiene otro nombre debe especificarse en `configuraciones_generales.yaml` en el parámetro `nombre_archivo_paradas`.

Es importante aclarar que en la medida en que se necesita una única línea por cada ramal, no se considera el sentido el recorrido (ida o vuelta). Se debe tomar uno solo para construir las paradas. En caso de que existan diferencias en el recorrido, se puede desviar el mismo para que pase por un punto medio y seguir siendo un recorrido representativo del ramal.

___

This notebook is a tool to create a set of stops as needed by `Urbantrips`, with a step order and a `node_id` (which is reaffirmed to build graphs. There are 2 non-exclusive situations. On the one hand, there is a cartography of routes of the different lines and branches and you want to establish stops using a regular interval in meters along that route.This situation is typical of bus routes.On the other hand, there is a cartography of stops and one of routes for the different lines and branches, for which you want to establish an order of passage. 

This notebook is nothing more than a helper in this task and facilitate the preparation of the dataset of stops. The final file will surely need custom and handmade editions with other criteria in mind. Final dataset must be named `stops.csv` and saved into `data/data_ciudad/`. If it has another name, it must be specified in `general_configurations.yaml` in the `stops_file_name` parameter.

It is important to clarify that `urbantrips` requires a single line for each branch. So the direction of the route  is not considered. Only one should be taken to build the stops. In the event that there are differences in the route, it can be redrawn so that it passes through a midpoint and continues to be a representative route of the branch.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from urbantrips.carto import stops, routes
from urbantrips.utils import utils
import networkx as nx
import osmnx as ox

Reemplace `PATH` con a ruta al directorio donde fue clonado el repositorio de `urbantrips` para que lea correctamente el archivo de configs.
___

Replace `PATH` with the path to the directory where the `urbantrips` repository was cloned so that it reads the configs file correctly.

In [ ]:
URBANTRIPS_PATH = "[PATH]"

os.chdir(URBANTRIPS_PATH)

configs = utils.leer_configs_generales()
geojson_path = os.path.join(URBANTRIPS_PATH,"data","data_ciudad",configs['recorridos_geojson'])


## 1. Proyectar paradas sobre recorridos - Project stops over routes


A partir de cada linea se establece una parada cada determinada cantidad de metros (definda por el parámetro `stops_distance` que debe estar como atributo en el geojson de los recorridos). A la hora de armar los recorridos como un grafo, y permitir transbordo dentro de ramales en una misma linea, se agregan paradas en mismo nodo. El criterio es agregar todas las paradas que esten a una determinada distancia entre sí. Este parámetro se fija parada cada `id_linea` en `line_stops_buffer` (que también debe estar como atributo en el geojson de los recorridos).

Este notebook sigue paso a paso este proceso y permite tunear el parámetro `line_stops_buffer` para cada linea, volviendo a guardar el resultado en el geojson de los recorridos.

El dataset de recorridos disponible en el repositorio es usado como ejemplo.
___

From each line, a stop is interpolated every certain number of meters (defined by the `stops_distance` parameter, which must be an attribute in the geojson of the routes). When building the routes as a graph, and allowing transfers within branches on the same line, stops are added at the same node. The criterion is to add all the stops that are at a certain distance from each other. This parameter sets each `id_linea` stop in `line_stops_buffer` (which must also be an attribute in the geojson of the routes).

This notebook follows this process step by step and allows you to tune the `line_stops_buffer` parameter for each line, saving the result back to the geojson of the tracks.

The track dataset available in the repository is used as an example.


In [ ]:
gdf = gpd.read_file(geojson_path)
gdf.head(3)

A partir de los recorridos y el parámetro presente en `stops_distance` en el dataset para cada linea/ramal, se crearon las siguientes paradas.

____

From the routes and the parameter present in `stops_distance` in the dataset for each line/branch, the following stops were created.

In [ ]:
stops_gdf = stops.create_line_stops_equal_interval(geojson_path)
stops_gdf.head(3)

Se puede chequear para cada linea la ubicación de las paradas interpoladas

___

You can check for each line the location of the interpolated stops

In [ ]:
stops_gdf.explore(column = 'id_linea', categorical = True,tiles="CartoDB positron",
                  marker_kwds = {'radius':10}, cmap='Set2')

En esta viz puede chequear el orden de las paradas en el ramal para ver que sea incremental y continuo.

____

In this viz you can check the order of the stops in the branch to see that it is incremental and continuous.

In [ ]:
id_ramal = 0
stops_gdf.query(f"id_ramal == {id_ramal}").explore(column = 'branch_stop_order',
                                                   categorical = True,tiles="CartoDB positron",
                                                   marker_kwds = {'radius':10})

Ahora se puede tunear el parámetro `line_stops_buffer` para cada linea. 

Lo ideal es que no se hayan eliminado demasiadas paradas ni que queden demasiadas superpuestas. 

Si hay muy pocas paradas y son demasiadas las que fueron fusionadas en un solo `node_id` entonces utilizar un 
`line_stops_buffer` más conservador, de menor valor.

Si son demasiadas paradas y se desea fusionar algunas más, elegir un `line_stops_buffer` mayor.

Para la linea `143` con 2 ramales (que sigue el recorrido de la linea 12 de CABA) ligeras diferencias en el recorrido hace que se interpolen paradas demasiado cerca entre si para los dos ramales. Si se utiliza `line_stops_buffer = 50` estan quedarán duplicadas, pero si se utiliza `line_stops_buffer = 100` se fusionaran en una sola parada, en especial donde ambos ramales se superponen.

___
You can now tune the `line_stops_buffer` parameter for each line.

Ideally, not too many stops have been removed or too many overlapping ones.

If there are too few stops and too many were merged into a single `node_id` then use a
`line_stops_buffer` more conservative, lower value.

If there are too many stops and you want to merge some more, choose a larger `line_stops_buffer`.

For line `143` with 2 branches (which follows the route of line 12 of CABA) slight differences in the route mean that stops are interpolated too close to each other for the two branches. If `line_stops_buffer = 50` is used they will be duplicated, but if `line_stops_buffer = 100` is used they will be merged into a single stop, especially where both branches overlap.

In [ ]:
# Elegir linea y parametros
id_linea = 143
line_stops_buffer = 100

line_stops_gdf = stops_gdf.loc[stops_gdf.id_linea == id_linea,:]
line_stops_gdf.loc[:,['line_stops_buffer']] = line_stops_buffer

# Agregar a node_id
stops_with_node_id = stops.create_node_id(line_stops_gdf)

# GDF para visualizar
geometry = gpd.points_from_xy(stops_with_node_id['node_x'], stops_with_node_id['node_y'], crs= 'EPSG:4326')
stops_with_node_id_gdf = gpd.GeoDataFrame(stops_with_node_id,geometry=geometry,crs = f"EPSG:4326")
stops_with_node_id_gdf.explore(color = 'red',tiles="CartoDB positron", marker_kwds = {'radius':10})

Si el parámetro es correcto se puede agregar al dataset y guardarlo en el mismo geojson

___

If the parameter is correct, it can be added to the dataset and saved in the same geojson

In [ ]:
geojson_data = gpd.read_file(geojson_path)
geojson_data.loc[geojson_data.id_linea == id_linea,['line_stops_buffer']] = line_stops_buffer
geojson_data.to_file(geojson_path, driver='GeoJSON')

Una vez que el parámetro `line_stops_buffer` ha sido satisfactoriamente agregado para cada `id_linea`, puede correr `stops.create_temporary_stops_csv_with_node_id(geojson_path)` que dejará guardado el dataset de paradas con orden y `node_id` con el nombre `temporary_stops.csv`. Si lo desea puede agregar otro set de paradas que se haya confeccionado manualmente, pero debe tener la misma esctructura. Esto puede ser util para cuando se decide crear de modo manual un dataset de estas características para modos como el metro o el tren. El archivo final para crear los grafos de lineas y ramales debe llamarse `stops.csv`.

___

Once the `line_stops_buffer` parameter has been successfully added for each `line_id`, you can run `stops.create_temporary_stops_csv_with_node_id(geojson_path)` which will save in`temporary_stops.csv` the stops dataset with order and `node_id`. If you wish, you can add another set of stops that has been created manually, but it must have the same structure. This can be useful when it is decided to manually create a dataset of these characteristics for modes such as the subway or the train. The final csv file to be used to create the line and branches graphs must be named `stops.csv`.


In [ ]:
stops.create_temporary_stops_csv_with_node_id(geojson_path)

## 2. Tomando paradas y recorridos, establecer orden de paso y node_id - From stops and routes, set order and node_id  


En algunas ocasiones, por ejemplo con modos guiados como trenes y metros, subtes, tranvías, normalmente se tiene una cartografía de paradas o estaciones junto con una de recorridos para las diferentes lineas y ramales. Lo que queda por hacer entonces para que `Urbantrips` pueda utilizarlas es, por un lado, establecer un orden de paso y, por el otro, fusionar dos estaciones donde se pueda hacer transbordo entre ramales de una misma linea, asignándoles un mismo `node_id` que luego será utilizado en el grafo. Como para esto es necesario establecer un parametro de cercania en metros, se utilizará la CRS en metros especificada en `configs`. En esta parte del notebook veremos como realizar esta tarea. 
___

On some occasions, for example with guided modes such as trains and subways, subways, trams, there is normally a cartography of stops or stations together with one of routes for the different lines and branches. What remains to be done then so that `Urbantrips` can use them is, on the one hand, to establish an order of passage and, on the other, to merge two stations where it is possible to transfer between branches of the same line, assigning them the same `node_id ` which will then be used in the graph. As for this it is necessary to establish a proximity parameter in meters, the CRS in meters specified in `configs` will be used.  In this part of the notebook we will see how to perform this task.

In [ ]:
# some helper functions
def create_stops_with_branch_stop_order(stops_gdf,routes_gdf):
    # add route geom to the stop geom
    stops_with_order = stops_gdf.merge(routes_gdf,
                                       how = 'left',
                                       on = ['id_linea','id_ramal'],
                                      suffixes = ('_stops','_routes'))
    # project stop on route
    stops_with_order['order'] = stops_with_order.apply(project_stop,axis=1)
    
    # create a branch stop order
    branch_stop_order = stops_with_order.groupby(['id_linea','id_ramal']).apply(create_branch_stop_order)
    branch_stop_order.index = branch_stop_order.index.droplevel([0,1])
    stops_with_branch_order = stops_with_order.join(branch_stop_order)

    # set schema
    stops_with_branch_order = stops_with_branch_order\
        .reindex(columns = ['id_linea','id_ramal','branch_stop_order','geometry_stops'])\
        .rename(columns = {'geometry_stops':'geometry'})

    # turn into geodataframe
    stops_with_branch_order = gpd.GeoDataFrame(stops_with_branch_order, geometry = 'geometry', crs = stops_gdf.crs)
    
    return stops_with_branch_order


def project_stop(row):
    return row.geometry_routes.project(row.geometry_stops, normalized=True)

def create_branch_stop_order(df):
    df = df.sort_values('order')
    return pd.Series(range(len(df)), index = df.index, name = 'branch_stop_order')

Utilizaremos la misma cartografía de de paradas y de rutas utilizadas en el ejemplo anterior que forman parte de los tests de `Urbantrips`.

___

We will use the same cartography of stops and routes used in the previous example that are part of the `Urbantrips` tests.

In [ ]:
#read routes geoms
routes_gdf = gpd.read_file(geojson_path)
routes_gdf = routes_gdf.reindex(columns = ['id_linea','id_ramal','geometry'])

#read stops geoms
stops_gdf = stops.create_line_stops_equal_interval(geojson_path)
stops_gdf = stops_gdf.reindex(columns = ['id_linea','id_ramal','geometry'])

Como se puede ver se trata de 2 ramales diferentes, donde uno se extiende más que el otro. En el tramo en común, se superponen muchas paradas. 

___

These are 2 different branches, where one extends more than the other. In the common section, many stops overlap.

In [ ]:
stops_gdf\
    .query("id_linea == 143")\
    .explore(
        column = 'id_ramal',
        tiles="CartoDB positron",categorical = True,
        marker_kwds = {'radius':10},
        cmap='Paired'
    )

Se establecerá un orden de paso para cada parada en base al recorrido del ramal. Al visualizar utilizando el atributo `branch_stop_order` se puede ver el orden de paso creciente en sentido norte.

___

An order of passage will be established for each stop based on the route of the branch. When displaying using the `branch_stop_order` attribute you can see the increasing step order northbound

In [ ]:
stops_with_branch_order = create_stops_with_branch_stop_order(stops_gdf,routes_gdf)
stops_with_branch_order\
    .query("id_linea == 143")\
    .explore(
        column = 'branch_stop_order',
        tiles="CartoDB positron",
        marker_kwds = {'radius':10}
    )

Una vez establecido el orden de paso, es necesario fusionar las paradas que están demasiado cerca entre sí como para suponer que en realidad constituyen la misma parada. Para eso se establece un parametro de cercania en metros. El mismo puede ser el mismo para todas las lineas o variar para cada linea, para reflejar particularidades de cada una. Se agrega como atributo al dataset, lo mismo que las coordenadas.

____

Once the order of passage is established, it is necessary to merge stops that are too close to each other to assume that they are actually the same stop. For that, a proximity parameter is established in meters. The same can be the same for all the lines or vary for each line, to reflect special situations in each one. It is added as an attribute to the dataset, as the coordinates.

In [ ]:
line_stops_buffer = {
    143:100,
    48:80
}
stops_with_branch_order.loc[:,['line_stops_buffer']] = stops_with_branch_order.id_linea.replace(line_stops_buffer)


stops_with_node_id = stops.aggregate_line_stops_to_node_id(stops_with_branch_order)

Convertimos el DataFrame en un GeoDataFrame para explorarlo utilizando el `node_id` para colorear.
 
___
 
Turn DataFrame into GeoDataFrame to explore with `node_id` for coloring.
 
 

In [ ]:
line_stops = stops_with_node_id.query("id_linea == 143").copy()

line_stops_gdf = gpd.GeoDataFrame(
    line_stops,
    geometry = gpd.GeoSeries.from_xy(x=line_stops.node_x,y=line_stops.node_y),
    crs = f"EPSG:4326")

line_stops_gdf.query("id_linea == 143").explore(column = 'node_id',
                                                      tiles="CartoDB positron",
                                            marker_kwds = {'radius':10})

Por último, para cada ramal se produce un grafo en base a los `node_id` y ordenados por el orden de paso del ramal. Luego, se fusionan todos los grafos del ramal en un grafo único de la linea. Esto permite que cuando un usuario se suba en un ramal de la linea (por ejemplo un ramal del metro o subte) y se baje en una estación de otro ramal (sin que las transacciones en la tarjeta marquen este transbordo) se pueda computar una distancia recorrida y una ruta de viaje dentro de la linea.

____


Finally, for each branch, a graph is produced based on the `node_id` and ordered by the order of passage of the branch. Then, all branch's graphs are merged into a single line graph. So when a user gets on a branch of the line (for example, a metro or subway branch) and gets off at a station on another branch (without card transactions detecting this transfer), a distance traveled in meters can be computed as well as a travel route within the line.

In [ ]:
branches_id = line_stops.id_ramal.unique()

G_line = nx.compose_all([routes.create_branch_g_from_stops_df(
        line_stops, branch_id) for branch_id in branches_id])

fig, ax = ox.plot_graph(G_line, save=False)

Esto puede verse mucho mejor utilizando la otra linea con más ramales que se bifurcan. Si se simula un viaje con un origen y un destino se puede rutear a lo largo del grafo, obteniendo la ruta y la distancia recorrida.

Justamente por tener más ramales se utilizó un parámetro de cercanía menor (`line_stops_buffer`), porque al haber tantos ramales y tantas paradas demasiado cerca, podría fusionar demasiadas paradas en una sola, sobre-simplificando el grafo. Las paradas de esta linea seguramente se vean beneficiadas de algunos ajustes manuales para que el grafo se arme de la mejor manera.
___


This can be seen much better using the other line with more branches that branch off. If a trip with an origin and a destination is simulated, it can be routed along the graph, obtaining the route and the distance traveled.

Precisely because it has more branches, a smaller proximity parameter (`line_stops_buffer`) was used, because having so many branches and so many stops too close, it could merge too many stops, over-simplifying the graph. The stops on this line will surely benefit from some manual adjustments so that the graph is assembled in the best way.



In [ ]:
line_stops = stops_with_node_id.query(f"id_linea == 48").copy()
branches_id = line_stops.id_ramal.unique()

G_line = nx.compose_all([routes.create_branch_g_from_stops_df(
        line_stops, branch_id) for branch_id in branches_id])

o_y,o_x = -34.573455, -58.439341
d_y,d_x = -34.617003, -58.379984

orig = ox.distance.nearest_nodes(G_line, X=o_x, Y=o_y)
dest = ox.distance.nearest_nodes(G_line, X=d_x, Y=d_y)  

route = ox.shortest_path(G_line, orig, dest, weight="length")
fig, ax = ox.plot_graph_route(G_line, route, node_size=0)


edge_lengths = ox.utils_graph.get_route_edge_attributes(G_line, route, "length")
print(f"El viaje recorrió {round(sum(edge_lengths))} metros")

Si el resultado es satisfactorio o quiere usarse ese archivo csv para introducir cambios manuales puede guardarse.

___

If the result is satisfactory or you want to use that csv file to introduce manual changes it can be saved.

In [ ]:
temp_stops_csv_path = os.path.join(URBANTRIPS_PATH,"data","data_ciudad","temporary_stops_train_subway.csv")

stops_with_node_id.to_csv(temp_stops_csv_path,index=False)